# Branch and Bound Algorithm for Solving Mixed Integer Programs (MIPs) or Binary Integer Programs (BIPs)

- The idea of the Branch & Bound algorithm is to indirectly arrive at an integer solution by running multiple linear programs that do not have constraints on the integer variables.  Multiple constraints are successively added that in some cases force solutions to have integer decision variables.  
- Integer solutions thus obtained may or may not be an optimal solution because the constraints break the preceding model into two parts: the other subpart might have a better solution.  These integer solutions are, therefore, a lower bound (__bound__) on the best possible solution.
- Non-integer solutions for a subproblem are possible upper bounds on the best possible objective function value if the solution is less than previous upper bounds (__bound__).
- Breaking an existing problem into two with two mutually exclusive but exhaustive constraints is called __branching__.
- Termination of a branch: __pruning__
  - A branch does not need to investigated further if its solution, either integer or not, when its solution is less than the current lower bound
  - A branch does not need to be investigated further if the problem is infeasible: no other subproblems on that branch will be feasible.
- Termination of the algorithm
  - The optimal solution has been found when the current integer solution has an objective function (lower bound) greater than the solutions on all other branches (which, of course, have been pruned).ted
ted


# Linear Relaxation

The B&B Algorithm starts with solving a linear program without any integrality constraints on the integer decision variables.

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

A = [[4,3], [7,2], [0,1]]
b = [120, 160, 32]
c = [20, 10]
products = ['sofa', 'chair']

# Create Gurobi model
m = gp.Model('show_b_and_b')

''' Specify whether model is maximized or minimized   (model sense) '''
m.ModelSense = GRB.MAXIMIZE

# Create decision variables
x = [m.addVar(vtype=GRB.CONTINUOUS, name=f'x_{products[i]}') for i in range(len(c))]

# Update model to include variables
m.update()

# Create objective function
m.setObjective(gp.quicksum(c[i]*x[i] for i in range(len(c))))

# Create constraints
for i in range(len(b)):
    m.addConstr((gp.quicksum(A[i][j]*x[j] for j in range(len(A[i]))) <= b[i]), 
                name=f'constr_{i}')

# Update model to include constraints and objective function
m.update()

# Optimize the model
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.X}')

# Build a Function to Create/Run the Base LP Relaxation

We will need to run many optimizations based on this basic LP relaxation and so it makes sense to write a function that can build and return this model.  The function returns the Gurobi model (<code>m</code>) and the decision variables (<code>x</code>) that are needed to add constraints in the branch and bound algorithm.

In [ ]:
def run_lp_relaxation(A, b, c, name='fun'):
    import gurobipy as gp
    from gurobipy import GRB
    
    relax_A = A
    relax_b = b
    relax_c = c
    
    # Create Gurobi model
    m = gp.Model(name)
    
    # Specify how to optimize and time limit (seconds)
    m.setParam('TimeLimit',7200)

    # Specify whether model is maximized or minimized   (model sense) '''
    m.ModelSense = GRB.MAXIMIZE

    # Create decision variables
    x = m.addVars(len(relax_A[0]), vtype=GRB.CONTINUOUS, name='x')
    
    # Update model to include variables
    m.update()

    # Create objective function
    m.setObjective(gp.quicksum(relax_c[i]*x[i] for i in range(len(relax_c))))

    # Create constraints
    for i in range(len(relax_b)):
        m.addConstr((gp.quicksum(relax_A[i][j]*x[j] for j in range(len(relax_A[i]))) <= relax_b[i]), 
                name=f'constr_{i}')
    
    # Update model to include constraints and objective function
    m.update()

    return m,x

In [ ]:
m,x = run_lp_relaxation(A, b, c)

In [ ]:
m,x

In [ ]:
m.optimize()
''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.X}')

# Branch 1 on $x_s$

## Add Constraint: $x_s \leq 18$

In [ ]:
m,x = run_lp_relaxation(A, b, c)

m.addConstr(x[0] <= 18)
m.update()
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.X}')

## Add Constraint: $x_s \geq 19$

In [ ]:
m,x = run_lp_relaxation(A, b, c)

m.addConstr(x[0] >= 19)
m.update()
m.optimize()

''' Print decision variable values and other information '''
for var in m.getVars():
    print(f'Variable: {var.varName}, Optimal Value = {var.X}')

# Running Large MIPs/BIPs

Running a larger MIP model likely involves one of two strategies for stopping the algorithm in a reasonable amount of time:

- Setting an explicit time limit
- Setting a percentage difference from the best possible objective function value
  - Gurobi will stop the optimization when this termination threshold has been met or exceeded (when % deviation less than the threshold)